In [7]:
from DataSplit import load_data
from Tokenizer import tokenize

X_train, X_eval, X_test, y_train, y_eval, y_test = load_data()


tokenize(X_train) # Basic tokenizer that removes punctuation and lowercases everything
tokenize(X_eval)
tokenize(X_test)


In [ ]:
from Tokenizer import remove_stopwords_and_stem
from Tokenizer import remove_stopwords_and_lemmatize
from FrequencyCut import cut
from PoSTagging import run
import numpy as np

# edit the stopwords to not remove the word "not"

for x in range(len(X_train)):
    # X_train[x] = remove_stopwords_and_stem(X_train[x])
    X_train[x] = remove_stopwords_and_lemmatize(X_train[x])

for x in range(len(X_eval)):
    # X_eval[x] = remove_stopwords_and_stem(X_eval[x])
    X_eval[x] = remove_stopwords_and_lemmatize(X_eval[x])

for x in range(len(X_test)):
    # X_test[x] = remove_stopwords_and_stem(X_test[x])
    X_test[x] = remove_stopwords_and_lemmatize(X_test[x])



# X_train = cut(X_train, 0.0001) #uncomment for frequency cut
# X_eval = cut(X_eval, 0.0001) #uncomment for frequency cut
# X_test = cut(X_test, 0.0001) #uncomment for frequency cut
# nounPhrasestrain = run(X_train) #uncomment this for noun phrases
# nounPhraseseval = run(X_eval) #uncomment this for noun phrases
# nounPhrasestest = run(X_test) #uncomment this for noun phrases






In [9]:
#Generate n-grams
from NGrams import generate_ngrams
n = 2
X_train_ngrams = generate_ngrams(X_train, n)
X_eval_ngrams = generate_ngrams(X_eval, n)
X_test_ngrams = generate_ngrams(X_test, n)

print (X_train_ngrams[1])


# Print the n-grams
# for i, ngrams in enumerate(X_train_ngrams):
#     print(f"Sentence {i+1} {n}-grams: {ngrams}")



[('folk', 'texaslouisiana'), ('texaslouisiana', 'border'), ('border', 'paramounts'), ('paramounts', 'b'), ('b', 'unit'), ('unit', 'inflicted'), ('inflicted', 'public'), ('public', 'area'), ('area', 'sabine'), ('sabine', 'river'), ('river', 'open'), ('open', 'cotton'), ('cotton', 'country'), ('country', 'deep'), ('deep', 'south'), ('south', 'deep'), ('deep', 'south'), ('south', 'post'), ('post', 'civl'), ('civl', 'cattle'), ('cattle', 'empire'), ('empire', 'theyre'), ('theyre', 'farther'), ('farther', 'west'), ('west', 'texas'), ('texas', 'farther'), ('farther', 'richard'), ('richard', 'dix'), ('dix', 'preston'), ('preston', 'foster'), ('foster', 'ride'), ('ride', 'empirebr'), ('empirebr', 'partner'), ('partner', 'riverboat'), ('riverboat', 'leo'), ('leo', 'carrillo'), ('carrillo', 'theft'), ('theft', 'service'), ('service', 'paying'), ('paying', 'hauling'), ('hauling', 'cattle'), ('cattle', 'cattle'), ('cattle', 'ponderosa'), ('ponderosa', 'ranch'), ('ranch', 'startbr'), ('startbr', 'f

In [ ]:
#Feature extraction
import pandas as pd
import numpy as np
from termfrequency import collect_vocabulary, get_terms, vectorize, calculate_idfs, vectorize_idf
# get term frequency for the tokens

doc_terms = {}
for doc_id in range(len(X_train)):
    doc_terms[doc_id] = get_terms(X_train[doc_id]# apply to the content of the document with id doc_id
                                  )
    
# collect the vocabulary from the training set
allvocab = collect_vocabulary(X_train)
doc_vectors = vectorize(doc_terms,allvocab # apply vectorize to the doc_terms and the shared vocabulary allvocab
                        )
doc_idfs = calculate_idfs(allvocab, doc_terms# apply calculate_idfs to the shared vocabulary allvoccab and to doc_terms
                        )
doc_vectors = vectorize_idf(doc_terms, doc_idfs, allvocab)
df_Train = pd.DataFrame.from_dict(doc_vectors, orient='index', columns=allvocab)

print("done1")

# do the same for the eval set
doc_terms_eval = {}
for doc_id in range(len(X_eval)):
    doc_terms_eval[doc_id] = get_terms(X_eval[doc_id]# apply to the content of the document with id doc_id
                                  )
allvocab = collect_vocabulary(X_eval)
doc_vectors = vectorize(doc_terms_eval,allvocab # apply vectorize to the doc_terms and the shared vocabulary all_terms
                        )
doc_idfs = calculate_idfs(allvocab, doc_terms_eval# apply calculate_idfs to the shared vocabulary all_terms and to doc_terms
                        )
doc_vectors = vectorize_idf(doc_terms_eval, doc_idfs, allvocab)
df_eval = pd.DataFrame.from_dict(doc_vectors, orient='index', columns=allvocab)

print("done2")

doc_terms_test = {}
for doc_id in range(len(X_test)):
    doc_terms_test[doc_id] = get_terms(X_test[doc_id]# apply to the content of the document with id doc_id
                                  )
allvocab = collect_vocabulary(X_test)
doc_vectors = vectorize(doc_terms_test,allvocab) # apply vectorize to the doc_terms and the shared vocabulary all_terms
doc_idfs = calculate_idfs(allvocab, doc_terms_test) # apply calculate_idfs to the shared vocabulary all_terms and to doc_terms
doc_vectors = vectorize_idf(doc_terms_test, doc_idfs, allvocab)
df_test = pd.DataFrame.from_dict(doc_vectors, orient='index', columns=allvocab)
print("donetest")

# TF-IDF for n-grams training set
# doc_terms_ngrams_train = {}
# for doc_id in range(len(X_train_ngrams)):
#     doc_terms_ngrams_train[doc_id] = get_terms(X_train_ngrams[doc_id]# apply to the content of the document with id doc_id
#                                   )
# allvocab = collect_vocabulary(X_train_ngrams)
# doc_vectors = vectorize(doc_terms_ngrams_train,allvocab # apply vectorize to the doc_terms and the shared vocabulary all_terms
# )
# doc_idfs = calculate_idfs(allvocab, doc_terms_ngrams_train# apply calculate_idfs to the shared vocabulary all_terms and to doc_terms
#                         )
# doc_vectors = vectorize_idf(doc_terms_ngrams_train, doc_idfs, allvocab)

# allvocabstr = ['_'.join(col) for col in allvocab]
# df_train_ngrams = pd.DataFrame.from_dict(doc_vectors, orient='index', columns=allvocabstr)

# print("done3")

# # TF-IDF for n-grams eval set
# doc_terms_ngrams_eval = {}
# for doc_id in range(len(X_eval_ngrams)):
#     doc_terms_ngrams_eval[doc_id] = get_terms(X_eval_ngrams[doc_id]# apply to the content of the document with id doc_id
#                                   )
# allvocab = collect_vocabulary(X_eval_ngrams)
# doc_vectors = vectorize(doc_terms_ngrams_eval,allvocab # apply vectorize to the doc_terms and the shared vocabulary all_terms
# )
# doc_idfs = calculate_idfs(allvocab, doc_terms_ngrams_eval# apply calculate_idfs to the shared vocabulary all_terms and to doc_terms
#                         )
# doc_vectors = vectorize_idf(doc_terms_ngrams_eval, doc_idfs, allvocab)
# allvocabstr = ['_'.join(col) for col in allvocab]
# df_eval_ngrams = pd.DataFrame.from_dict(doc_vectors, orient='index', columns=allvocabstr)

# print("done4")

# # TF-IDF for n-grams test set
# doc_terms_ngrams_test = {}
# for doc_id in range(len(X_test_ngrams)):
#     doc_terms_ngrams_test[doc_id] = get_terms(X_test_ngrams[doc_id]# apply to the content of the document with id doc_id
#                                   )
# allvocab = collect_vocabulary(X_test_ngrams)
# doc_vectors = vectorize(doc_terms_ngrams_test,allvocab # apply vectorize to the doc_terms and the shared vocabulary all_terms
# )
# doc_idfs = calculate_idfs(allvocab, doc_terms_ngrams_test# apply calculate_idfs to the shared vocabulary all_terms and to doc_terms
#                         )
# doc_vectors = vectorize_idf(doc_terms_ngrams_test, doc_idfs, allvocab)
# allvocabstr = ['_'.join(col) for col in allvocab]
# df_test_ngrams = pd.DataFrame.from_dict(doc_vectors, orient='index', columns=allvocabstr)

# print("done5")


# doc_terms_nounphrsestrain = {}
# for doc_id in range(len(nounPhrasestrain)):
#     doc_terms_nounphrsestrain[doc_id] = get_terms(nounPhrasestrain[doc_id]# apply to the content of the document with id doc_id
#                                   )
# allvocab = collect_vocabulary(nounPhrasestrain)
# doc_vectors = vectorize(doc_terms_nounphrsestrain,allvocab # apply vectorize to the doc_terms and the shared vocabulary all_terms
#                         )
# doc_idfs = calculate_idfs(allvocab, doc_terms_nounphrsestrain# apply calculate_idfs to the shared vocabulary all_terms and to doc_terms
#                         )
# doc_vectors = vectorize_idf(doc_terms_nounphrsestrain, doc_idfs, allvocab)
# df_nounPhrasestrain = pd.DataFrame.from_dict(doc_vectors, orient='index', columns=allvocab)

# print("done6")
# doc_terms_nounphrseseval = {}
# for doc_id in range(len(nounPhraseseval)):
#     doc_terms_nounphrseseval[doc_id] = get_terms(nounPhraseseval[doc_id]# apply to the content of the document with id doc_id
#                                   )
# allvocab = collect_vocabulary(nounPhraseseval)
# doc_vectors = vectorize(doc_terms_nounphrseseval,allvocab # apply vectorize to the doc_terms and the shared vocabulary all_terms
#                         )
# doc_idfs = calculate_idfs(allvocab, doc_terms_nounphrseseval# apply calculate_idfs to the shared vocabulary all_terms and to doc_terms
#                         )
# doc_vectors = vectorize_idf(doc_terms_nounphrseseval, doc_idfs, allvocab)
# df_nounPhraseseval = pd.DataFrame.from_dict(doc_vectors, orient='index', columns=allvocab)

# print("done7")

# doc_terms_nounphrasestest = {}
# for doc_id in range(len(nounPhrasestest)):
#     doc_terms_nounphrasestest[doc_id] = get_terms(nounPhrasestest[doc_id]# apply to the content of the document with id doc_id
#                                   )
# allvocab = collect_vocabulary(nounPhrasestest)
# doc_vectors = vectorize(doc_terms_nounphrasestest,allvocab # apply vectorize to the doc_terms and the shared vocabulary all_terms
#                         )
# doc_idfs = calculate_idfs(allvocab, doc_terms_nounphrasestest# apply calculate_idfs to the shared vocabulary all_terms and to doc_terms
#                         )
# doc_vectors = vectorize_idf(doc_terms_nounphrasestest, doc_idfs, allvocab)
# df_nounPhrasestest = pd.DataFrame.from_dict(doc_vectors, orient='index', columns=allvocab)








done1
done2
donetest
done3
done4
done5


In [ ]:
from Normalizer import l2normalize, minmaxnormalize
def formatting(df,df2):
    # Ensure the DataFrame is in the correct order
    df = df.sort_index(axis=1)

    # Ensure the DataFrame is in the correct order
    df2 = df2.sort_index(axis=1)

    # Ensure df_eval has the same columns as df_Train
    # Remove columns not in df_Train
    df2 = df2.loc[:, df2.columns.isin(df.columns)]

    # Add missing columns to df_eval and set their values to 0
    missing_cols = [col for col in df.columns if col not in df2.columns]
    missing_df = pd.DataFrame(0, index=df2.index, columns=missing_cols)
    df2 = pd.concat([df2, missing_df], axis=1)

    # Reorder columns to match df_Train
    df2 = df2.reindex(columns=df.columns)

    return df, df2

def add_prefix_to_columns(df, prefix):
    df.columns = [f"{prefix}_{col}" for col in df.columns]
    return df

df_Train = l2normalize(df_Train)
df_eval = l2normalize(df_eval)
df_test = l2normalize(df_test)
# df_nounPhrasestrain = l2normalize(df_nounPhrasestrain)
# df_nounPhraseseval = l2normalize(df_nounPhraseseval)
# df_nounPhrasestest = l2normalize(df_nounPhrasestest)
# df_train_ngrams = l2normalize(df_train_ngrams)
# df_eval_ngrams = l2normalize(df_eval_ngrams)
# df_test_ngrams = l2normalize(df_test_ngrams)
# add_prefix_to_columns(df_nounPhrasestrain, "NP")
# add_prefix_to_columns(df_nounPhraseseval, "NP")
# add_prefix_to_columns(df_nounPhrasestest, "NP")
# add_prefix_to_columns(df_train_ngrams, "NG")
# add_prefix_to_columns(df_eval_ngrams, "NG")
# add_prefix_to_columns(df_test_ngrams, "NG")
# df_Train = pd.concat([df_Train, df_nounPhrasestrain], axis=1)
# df_eval = pd.concat([df_eval, df_nounPhraseseval], axis=1)
# df_test = pd.concat([df_test, df_nounPhrasestest], axis=1)
# df_Train = pd.concat([df_Train, df_train_ngrams], axis=1)
# df_eval = pd.concat([df_eval, df_eval_ngrams], axis=1)
# df_test = pd.concat([df_test, df_test_ngrams], axis=1)

df_Train, df_eval = formatting(df_Train, df_eval)
df_Train, df_test = formatting(df_Train, df_test)

In [12]:
#training the model
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score
from NaiveBayes import create_classifier

#to run on my own model
# model = create_classifier(df_Train.values, y_train)
# y_pred_eval = model.predict(df_test.values)
# accuracy_test = accuracy_score(y_test, y_pred_eval)
# print(f"Test Accuracy: {accuracy_test}")

#to run on the sklearn model

model = MultinomialNB()
model.fit(df_Train, y_train)
y_pred_eval = model.predict(df_eval)
accuracy_eval = accuracy_score(y_eval, y_pred_eval)
y_pred_test = model.predict(df_test)
accuracy_test = accuracy_score(y_test, y_pred_test)
print(f"Eval Accuracy: {accuracy_eval}")
print(f"Test Accuracy: {accuracy_test}")


Eval Accuracy: 0.8575
Test Accuracy: 0.8325
